In [1]:
1+1*9

10

In [2]:
%cd C:\Users\thoir\TCC python\Sales tracker

C:\Users\thoir\TCC python\Sales tracker


C:\Users\thoir\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
!git add .


fatal: not a git repository (or any of the parent directories): .git
